In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.models.ArcI.get_default_preprocessor(
    filter_mode='df',
    filter_low_freq=1,
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/93 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.342 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 93/93 [00:03<00:00, 29.54it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 92/92 [00:00<00:00, 135.37it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 9

In [4]:
preprocessor.context

{'embedding_input_dim': 336,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x27bc42afb00>,
 'vocab_size': 336,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x27bd201e7f0>}

In [5]:
# 由于加载glove模型会导致本案例运行速度会变慢，所以不加载glove模型

In [6]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

In [8]:
padding_callback = mz.models.ArcI.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=100,
    pad_word_value=0,
    pad_word_mode='pre'
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [9]:
model = mz.models.ArcI()

model.params['task'] = ranking_task
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['left_filters'] = [128]
model.params['left_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [4]
model.params['right_filters'] = [128]
model.params['right_kernel_sizes'] = [3]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.9

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcI(
  (embedding): Embedding(336, 100, padding_idx=0)
  (conv_left): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (conv_right): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.9, inplace=False)
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3456, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  456459


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-1 Loss-1.029]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3429 - normalized_discounted_cumulative_gain@5(0.0): 0.3429 - mean_average_precision(0.0): 0.3387



[Iter-2 Loss-0.958]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-3 Loss-0.828]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-4 Loss-0.718]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-5 Loss-0.458]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-6 Loss-0.305]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-7 Loss-0.246]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-8 Loss-0.051]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-9 Loss-0.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333



[Iter-10 Loss-0.141]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3387 - normalized_discounted_cumulative_gain@5(0.0): 0.3387 - mean_average_precision(0.0): 0.3333

Cost time: 5.1840269565582275s


In [12]:
import gc
gc.collect()

953